In [1]:
# GET DATA
import pandas as pd
import numpy as np
from get_data import get_assets, get_positional_data

get_assets()
positions = get_positional_data()


This process will pip install Kaggle and download data through Kaggle API.

Please confirm that you've downloaded Kaggle JSON credentials into directory

Data Successfully Downloaded


In [2]:
positions.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,week
0,2018-09-07T01:07:14.599Z,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,Matt Ryan,2.0,QB,1,away,2018090600,75,left,NaN,1
1,2018-09-07T01:07:14.599Z,88.89,36.47,0.01,0.01,0.01,105.63,66.66,None,79848.0,Malcolm Jenkins,27.0,SS,1,home,2018090600,75,left,NaN,1
2,2018-09-07T01:07:14.599Z,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,2018090600,75,left,HITCH,1
3,2018-09-07T01:07:14.599Z,86.31,22.01,0.09,0.42,0.01,70.12,168.91,None,2495613.0,Corey Graham,24.0,FS,1,home,2018090600,75,left,NaN,1
4,2018-09-07T01:07:14.599Z,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,2018090600,75,left,HITCH,1


In [3]:
import form_pred
import importlib
importlib.reload(form_pred)
from form_pred import clean_positional

clean_pos = clean_positional(positions)

In [4]:
clean_pos.head()

,CBR0_x,DER0_x,DLL0_x,DLR0_x,DTL0_x,FBL0_x,FBR0_x,FSL0_x,HBL0_x,HBL1_x,...,WRL1_in,WRL2_in,WRL3_in,WRR0_in,WRR1_in,WRR2_in,WRR3_in,gameId,playId,offenseFormation
0,0.0,0.0,0.0,0.0,0.0,-5.12,0.00,0.0,0.0,0.0,...,True,False,False,False,False,False,False,2018090600,75,I_FORM
6,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,False,False,False,True,True,False,False,2018090600,146,SINGLEBACK
12,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,False,False,False,True,False,False,False,2018090600,168,SHOTGUN
18,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,False,False,False,True,True,True,False,2018090600,190,SHOTGUN
24,0.0,0.0,0.0,0.0,0.0,0.00,-4.59,0.0,0.0,0.0,...,False,False,False,False,False,False,False,2018090600,256,SHOTGUN


In [5]:
import ball_movement
import importlib
importlib.reload(ball_movement)
from ball_movement import ball_quadrants, make_quad_chart

quads = ball_quadrants(positions,4)

make_quad_chart(quads)

alt.Chart(...)

In [7]:
import def_clean
import importlib
importlib.reload(def_clean)
from def_clean import *

weeks =  range(1,15)

import time
start_time = time.time()

#week_data = pd.read_csv('Kaggle-Data-Files/week1.csv')
print('.......reading inputs.......')
play_data = pd.read_csv('Kaggle-Data-Files/plays.csv')
weeks_data = pd.read_csv('assets/full_position.csv')
print('')
print('.......starting processing.......')
output_df = pd.DataFrame()

for week in weeks:
    week_data = weeks_data[weeks_data['week']==week]
    print('...Week {} loaded...'.format(str(week)))
    filtered_df = filter_full_position_df(week_data, 11)
    print('...filtered...')
    transform_df = transform_directions(filtered_df, play_data)
    print('...transformed...')
    full_df, reduced_df = reduce_time(transform_df, 11)
    #print('...reduced...')
    simple_df = simple_closest_player(full_df, reduced_df)
    #print('...measured...')
    start_df = starting_pos(full_df)
    action_df = generate_action_type_df(simple_df)
    total_df =  pd.concat([action_df,start_df],axis=0)
    total_df['week'] = week
    print('...generated...')
    if output_df.empty:
        output_df = total_df.copy(deep=True)
    else:
        output_df = output_df.append(total_df)

    print('.....Week {} COMPLETE.....'.format(str(week)))
    print('')
    percent_complete = round(week / len(weeks)*100,2)
    print('   {}% COMPLETE   '.format(str(percent_complete)))
    print('')
    end_time = time.time()
    print("--- {} minutes elapsed ---".format(round((end_time - start_time)/60,1)))
    print('')

    print("the weeks complete: ", output_df.week.unique())

output_df = output_df.pivot(index=['gameId', 'playId'], columns='posId',values='value')
output_df.to_csv('assets/def_clean_output.csv')

..............................initializing
.......reading inputs.......

.......starting processing.......
...Week 1 loaded...
...filtered...
...transformed...
...generated...
.....Week 1 COMPLETE.....

   7.14% COMPLETE   

--- 1.4 minutes elapsed ---

the weeks complete:  [1]
...Week 2 loaded...
...filtered...
...transformed...
...generated...
.....Week 2 COMPLETE.....

   14.29% COMPLETE   

--- 2.6 minutes elapsed ---

the weeks complete:  [1 2]
...Week 3 loaded...
...filtered...
...transformed...
...generated...
.....Week 3 COMPLETE.....

   21.43% COMPLETE   

--- 3.7 minutes elapsed ---

the weeks complete:  [1 2 3]
...Week 4 loaded...
...filtered...
...transformed...
...generated...
.....Week 4 COMPLETE.....

   28.57% COMPLETE   

--- 4.7 minutes elapsed ---

the weeks complete:  [1 2 3 4]
...Week 5 loaded...
...filtered...
...transformed...
...generated...
.....Week 5 COMPLETE.....

   35.71% COMPLETE   

--- 5.7 minutes elapsed ---

the weeks complete:  [1 2 3 4 5]
...Week 6

In [11]:
d = pd.read_csv('assets/def_clean_output.csv')
d.head()

,gameId,playId,CBL0_act,CBL0_x_start,CBL0_y_start,CBL1_act,CBL1_x_start,CBL1_y_start,CBL2_act,CBL2_x_start,...,SSL2_y_start,SSR0_act,SSR0_x_start,SSR0_y_start,SSR1_act,SSR1_x_start,SSR1_y_start,SSR2_act,SSR2_x_start,SSR2_y_start
0,2018090600,75,M,83.92,43.44,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018090600,146,M,41.61,39.97,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018090600,168,M,45.52,43.81,NaN,NaN,NaN,NaN,NaN,...,NaN,M,46.43,20.74,NaN,NaN,NaN,NaN,NaN,NaN
3,2018090600,190,Z,43.91,37.25,NaN,NaN,NaN,NaN,NaN,...,NaN,M,45.69,22.36,NaN,NaN,NaN,NaN,NaN,NaN
4,2018090600,256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
weeks =  range(15,18)

import time
start_time = time.time()

#week_data = pd.read_csv('Kaggle-Data-Files/week1.csv')
print('.......reading inputs.......')
play_data = pd.read_csv('Kaggle-Data-Files/plays.csv')
weeks_data = pd.read_csv('assets/full_position.csv')
print('')
print('.......starting processing.......')
output_df = pd.DataFrame()

for week in weeks:
    week_data = weeks_data[weeks_data['week']==week]
    print('...Week {} loaded...'.format(str(week)))
    filtered_df = filter_full_position_df(week_data, 11)
    print('...filtered...')
    transform_df = transform_directions(filtered_df, play_data)
    print('...transformed...')
    full_df, reduced_df = reduce_time(transform_df, 11)
    #print('...reduced...')
    simple_df = simple_closest_player(full_df, reduced_df)
    #print('...measured...')
    start_df = starting_pos(full_df)
    action_df = generate_action_type_df(simple_df)
    total_df =  pd.concat([action_df,start_df],axis=0)
    total_df['week'] = week
    print('...generated...')
    if output_df.empty:
        output_df = total_df.copy(deep=True)
    else:
        output_df = output_df.append(total_df)

    print('.....Week {} COMPLETE.....'.format(str(week)))
    print('')
    percent_complete = round(week / len(weeks)*100,2)
    print('   {}% COMPLETE   '.format(str(percent_complete)))
    print('')
    end_time = time.time()
    print("--- {} minutes elapsed ---".format(round((end_time - start_time)/60,1)))
    print('')

    print("the weeks complete: ", output_df.week.unique())

output_df = output_df.pivot(index=['gameId', 'playId'], columns='posId',values='value')
output_df.to_csv('assets/def_clean_output_test.csv')

.......reading inputs.......

.......starting processing.......
...Week 15 loaded...
...filtered...
...transformed...
...generated...
.....Week 15 COMPLETE.....

   500.0% COMPLETE   

--- 1.6 minutes elapsed ---

the weeks complete:  [15]
...Week 16 loaded...
...filtered...
...transformed...
...generated...
.....Week 16 COMPLETE.....

   533.33% COMPLETE   

--- 2.6 minutes elapsed ---

the weeks complete:  [15 16]
...Week 17 loaded...
...filtered...
...transformed...
...generated...
.....Week 17 COMPLETE.....

   566.67% COMPLETE   

--- 3.5 minutes elapsed ---

the weeks complete:  [15 16 17]


In [15]:
d_test = pd.read_csv('assets/def_clean_output_test.csv')
d_test.head()

,gameId,playId,CBL0_act,CBL0_x_start,CBL0_y_start,CBL1_act,CBL1_x_start,CBL1_y_start,CBL2_act,CBL2_x_start,...,SSL2_y_start,SSR0_act,SSR0_x_start,SSR0_y_start,SSR1_act,SSR1_x_start,SSR1_y_start,SSR2_act,SSR2_x_start,SSR2_y_start
0,2018121300,84,M,79.80,33.61,NaN,NaN,NaN,NaN,NaN,...,NaN,M,82.51,21.88,NaN,NaN,NaN,NaN,NaN,NaN
1,2018121300,111,M,63.94,23.80,M,63.60,13.45,NaN,NaN,...,NaN,Z,65.66,30.16,NaN,NaN,NaN,NaN,NaN,NaN
2,2018121300,164,NaN,NaN,NaN,M,90.85,13.86,NaN,NaN,...,NaN,M,100.06,30.25,NaN,NaN,NaN,NaN,NaN,NaN
3,2018121300,189,NaN,NaN,NaN,M,96.79,13.41,NaN,NaN,...,NaN,M,106.01,31.68,M,95.1,24.63,NaN,NaN,NaN
4,2018121300,211,NaN,NaN,NaN,Z,95.04,12.96,NaN,NaN,...,NaN,Z,107.17,29.89,NaN,NaN,NaN,NaN,NaN,NaN
